In [0]:
import numpy as np
from matplotlib import pyplot
from keras.optimizers import Adam
from keras.models import Sequential
from keras.utils.vis_utils import plot_model
from keras.datasets.cifar10 import load_data
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import Dense
from keras.layers import LeakyReLU
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Reshape

#discriminator model
def discriminator(in_shape=(32,32,3)):
  model = Sequential()

  #normal
  model.add(Conv2D(64, (3,3), padding='same', input_shape=in_shape))
  model.add(LeakyReLU(alpha=0.2))

  #downsampling with 2x2 stride 
  model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
  model.add(LeakyReLU(alpha=0.2))

  #downsampling with 2x2 stride 
  model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
  model.add(LeakyReLU(alpha=0.2))

  #downsampling with 2x2 stride 
  model.add(Conv2D(256, (3,3), strides=(2,2), padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  
  #classifier
  model.add(Flatten())
  model.add(Dropout(0.4))
  model.add(Dense(1, activation='linear'))
  
  #compile model
  opt = Adam(lr=0.0002, beta_1=0.5)
  model.compile( loss='mean_squared_error', optimizer=opt, metrics=['accuracy'])
  
  return model

#generator model
def generator(latent_dim):
  model = Sequential()
  
  #image foundation 4x4
  n_node = 256*4*4
  model.add(Dense(n_node, input_dim=latent_dim))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Reshape((4,4, 256)))
  
  #upsampling with 2x2 stride 
  model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  
  #upsampling with 2x2 stride 
  model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
  model.add(LeakyReLU(alpha=0.2))

  #upsampling with 2x2 stride 
  model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  
  #output layer
  model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))

  return model

#combined generator and discriminator model
def gan(g_model, d_model):
  #making discriminator model not trainable, while training generator
  d_model.trainable = False
  
  model = Sequential()

  #add generator model
  model.add(g_model)

  #add discriminator model
  model.add(d_model)
  
  #compile them
  opt = Adam(lr=0.0002, beta_1=0.5)
  model.compile(loss='mean_squared_error', optimizer=opt)
  
  return model

#load images from cifar10 dataset
def load_real_samples():
  #load dataset
  (x_train, _),(_,_) = load_data()
  #convert unsigned int to float
  x = x_train.astype('float32')
  #scale from [0,255] to [-1,1]
  x = (x-127.5)/127.5
  return x

#select real images
def generate_real_samples(dataset, n_samples):
  #choose x random instances btw [0, no_of_records_in_dataset] where x = n_samples
  rn = np.random.randint(0, dataset.shape[0], n_samples)
  #retrieve selected images
  x = dataset[rn]
  #create 'real' class labels (1)
  y = np.ones((n_samples,1))
  return x,y

#generate points in latent space as input for generator
def generate_latent_points(latent_dim, n_samples):
  #generate points
  x_input = np.random.randn(latent_dim*n_samples)
  #reshape them into a batch of inputs
  x_input = x_input.reshape(n_samples, latent_dim)
  return x_input

#generate fake images using generator
def generate_fake_samples(g_model, latent_dim, n_samples):
  #generate points in latent space (random noise)
  x_input = generate_latent_points(latent_dim, n_samples)
  #feed these points (noise) to generator for fake image output
  x = g_model.predict(x_input)
  #create 'fake' class labels (0)
  y = np.zeros((n_samples,1))
  return x,y

#display fake images generated by generator
def save_plot(examples, epoch, n = 7):
  examples = (examples + 1)/2.0
  
  for i in range(n*n):
    pyplot.subplot(n, n, i+1)
    pyplot.axis('off')
    pyplot.imshow(examples[i])
    
  filename = "generated_plot_e{epoch+1}.png"
  pyplot.savefig(filename)
  pyplot.show()
  pyplot.close()

#evaluate discriminator and display fake images generated
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=150):
  #real images samples
  x_real, y_real = generate_real_samples(dataset, n_samples)
  #evaluate discriminator on real images
  _, accuracy_real = d_model.evaluate(x_real, y_real, verbose=0)
  
  #fake image samples generated by generator
  x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
  #evaluate discriminator on fake images
  _, accuracy_fake = d_model.evaluate(x_fake, y_fake, verbose=0)

  #summarize discriminator performance by accuracy
  print(f"Accuracy on real={accuracy_real*100}% & on fake={accuracy_fake*100}%")

  #display fake images generated by generator
  save_plot(x_fake, epoch)
  #save generator model
  filename = f"generator_model_{epoch+1}.h5"
  g_model.save(filename)

#plot loss in each iteration
def plot_history(d1_hist, d2_hist, g_hist):
  pyplot.plot(d1_hist, label='D_1oss1')
  pyplot.plot(d2_hist, label='D_1oss2')
  pyplot.plot(g_hist, label='G_1oss')
  pyplot.legend()
  filename = "loss_line_plot.png"
  pyplot.savefig(filename)
  pyplot.show()
  pyplot.close()

#train generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=50, n_batch=128):
  batch_per_epoch = int(dataset.shape[0]/n_batch)
  half_batch = int(n_batch/2)
  #lists for appending loss for each iteration
  d1_hist, d2_hist, g_hist = list(), list(), list()

  for i in range(n_epochs):
   
    for j in range(batch_per_epoch):
      #randomly selected real images
      x_real, y_real = generate_real_samples(dataset, half_batch)
      #training discriminator model for above real samples
      d_loss1,_ = d_model.train_on_batch(x_real, y_real)
      
      #generating fake samples
      x_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
      #training discriminator model for above fake samples
      d_loss2,_ = d_model.train_on_batch(x_fake, y_fake)
      
      #generating points in latent space as input to generator
      x_gan = generate_latent_points(latent_dim, n_batch)
      #creating inverted labels for fake images
      y_gan =np.ones((n_batch,1))
      #training the generator via discriminator's errors
      g_loss = gan_model.train_on_batch(x_gan, y_gan)

      #summarizing losses in this batch
      print(f"{i+1}--{j+1}/{batch_per_epoch}-- D1={d_loss1}, D2={d_loss2}, G={g_loss}")
      
      #appending above losses to loss list for plotting at the end
      d1_hist.append(d_loss1)
      d2_hist.append(d_loss2)
      g_hist.append(g_loss)

    #evaluating the model performance at intervals
    if (i+1)%10 == 0:
      summarize_performance(i, g_model, d_model, dataset, latent_dim)

  #plotting the losses recorded during complete training    
  plot_history(d1_hist, d2_hist, g_hist)

#latent space dimension
latent_dim = 100

#create discriminator
d_model = discriminator()

#create generator
g_model = generator(latent_dim)

#create gan
lsgan_model = gan(g_model, d_model)

#load real image data
dataset = load_real_samples()

#train model
train(g_model, d_model, lsgan_model, dataset, latent_dim)